# U-Net Experiement 1: Exploring variance/ randomness

When running the exact same model with the exact same dataset we get slightly different results each time due to randomness and variability involved in the training process.
the variance refers to 
The goal of this notebook is to evaluate the variability in the results obtained from different training runs on the same dataset We will use the exact same setup and dataset as in the unet notebook and execute 10 training runs.

### 0. Select Python version

In [ ]:
!sudo apt-get install python3.10
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3

### 1. Loading + Preparing Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')
! ls

Mounted at /content/drive
drive  sample_data


In [2]:
! git pull

fatal: not a git repository (or any of the parent directories): .git


In [2]:
%cd drive/MyDrive/MachineLearning/
#! git clone https://github.com/emely3h/Geospatial_ML.git
%cd Geospatial_ML
! ls

/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
architecture.drawio  evaluation   models	README.md
data_exploration     experiments  prepare_data	requirements.txt


In [3]:
from models.helpers import normalizing_encoding, define_model, train_model, save_model_history, make_predictions, evaluate_predictions, save_metrics
import numpy as np
import os
from models.unet_model import unet_2d
from models.helpers import get_mean_jaccard
from datetime import datetime

In [4]:
root = "../data_colab/256_200/"

y_train  = np.load(os.path.join(root,'2022_06_20.npz'))['y_mask']
x_train  = np.load(os.path.join(root,'2022_06_20.npz'))['x_input']

y_val = np.load(os.path.join(root,'2022_07_10.npz'))['y_mask']
x_val = np.load(os.path.join(root,'2022_07_10.npz'))['x_input']

y_test = np.load(os.path.join(root, '2022_07_25.npz'))['y_mask']
x_test = np.load(os.path.join(root, '2022_07_25.npz'))['x_input']

print(y_train.shape)
print(x_train.shape)

print(y_val.shape)
print(x_val.shape)

print(y_test.shape)
print(x_test.shape)

(1251, 256, 256)
(1251, 256, 256, 5)
(1323, 256, 256)
(1323, 256, 256, 5)
(1258, 256, 256)
(1258, 256, 256, 5)


In [ ]:
      # Get physical prediction
      train_physical
      val_physical
      test_physical
      # cut 5th channel
      # do labeling
      # one hot encodin

### 3. Normalizing + Encoding

In [5]:
x_train, y_train = normalizing_encoding(x_train, y_train)
X_val, y_val = normalizing_encoding(x_val, y_val)
x_test, y_test = normalizing_encoding(x_test, y_test)

y shape:  (1251, 256, 256)
Normalizing data...
y one hot shape:  (1251, 256, 256, 3)
y shape:  (1323, 256, 256)
Normalizing data...
y one hot shape:  (1323, 256, 256, 3)
y shape:  (1258, 256, 256)
Normalizing data...
y one hot shape:  (1258, 256, 256, 3)


### 4. Model summary

In [6]:
model = unet_2d(input_shape=(256, 256, 5), num_classes=3)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  2944        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

### 5. Execute trainings + saving results

In [7]:
tile_size = 256
step_size = 200
experiment = 'experiment_1'
training_dates = '2022_06_20'
validation_dates = '2022_07_10'
testing_dates = '2022_07_25'
all_metrics_test = []
all_metrics_val = []
all_metrics_train = []

In [8]:
def execute_multiple_trainings(num_of_executions):
  print(f'Start: {datetime.now()}')
  for count in range(0, num_of_executions):
      model = define_model((256, 256, 5), 3)
      print(f'Start training number {count}')
      
      # Train model
      history = train_model(model, x_train, y_train, x_val, y_val, epochs=1)
      print('training completed')
      
      # Save model
      model_name = f'{tile_size}_{step_size}_run_{count}'
      model.save(f'../models/{experiment}/model_{model_name}.h5')
      print('saving model completed')
      
      # Save model history
      save_model_history(history, model_name, f'../models/{experiment}')
      print('saving history completed')
      
      # Predict 
      pred_train, pred_val, pred_test = make_predictions(model, x_train, x_val, x_test)
      print('prediction completed')


      metrics_train = EvaluationMetrics(y_train, pred_train, train_physical)
      metrics_val = EvaluationMetrics(y_val, pred_val, val_physical)
      metrics_test = EvaluationMetrics(y_test, pred_test, test_physical)
      
      print('metrics calculated')
      
      # Save metrics
      save_metrics(metrics_train, metrics_val, metrics_test, saving_path, f'../metrics/{experiment}')

      print('saving metrics completed')
  
      print(f'Run {count} completed: {datetime.now()}')
    print(f'End: {datetime.now()}')
execute_multiple_trainings(1)

# executed with premium gpu and high ram => 13,08 computing units per hour, took

Start: 2023-04-03 22:39:17.378765
Start training number 0
Epoch 1/100
40/40 [==============================] - 42s 415ms/step - loss: 0.5176 - accuracy: 0.8350 - val_loss: 35.0789 - val_accuracy: 0.9527
Epoch 2/100
40/40 [==============================] - 10s 251ms/step - loss: 0.1119 - accuracy: 0.9659 - val_loss: 22.8870 - val_accuracy: 0.9667
Epoch 3/100
40/40 [==============================] - 10s 249ms/step - loss: 0.0892 - accuracy: 0.9716 - val_loss: 14.5426 - val_accuracy: 0.9726
Epoch 4/100
40/40 [==============================] - 10s 250ms/step - loss: 0.0756 - accuracy: 0.9733 - val_loss: 13.6730 - val_accuracy: 0.9748
Epoch 5/100
40/40 [==============================] - 10s 250ms/step - loss: 0.0671 - accuracy: 0.9756 - val_loss: 9.5149 - val_accuracy: 0.9769
Epoch 6/100
40/40 [==============================] - 10s 248ms/step - loss: 0.0616 - accuracy: 0.9770 - val_loss: 6.5805 - val_accuracy: 0.9794
Epoch 7/100
40/40 [==============================] - 10s 249ms/step - loss

TypeError: ignored

### 6. Results

In [ ]:
# load all 10 results
for idx, metric in enumerate(all_metrics_test):
  print(f'========= RUN {idx + 1} ============')
  print(f'TEST DATA')
  metric.print_metrics()
  print()
  print(f'VALIDATION DATA')
  all_metrics_val[idx].print_metrics()
  print(f'TRAINING DATA')
  all_metrics_train[idx].print_metrics()
  print()
  print()

In [ ]:
print('============ TEST DATA ===================')
get_mean_jaccard(all_metrics_test)
print()

print('============ VALIDATION DATA ===================')
get_mean_jaccard(all_metrics_val)
print()

print('============ TRAINING DATA ===================')
get_mean_jaccard(all_metrics_train)
print()